In [6]:
import pytorch_lightning as pl
from model import TargetModel, TargetType
from model import ShadowModel, ShadowType
from model import ReconstructModule

pl.seed_everything(38, workers=True)

Seed set to 38


38

In [7]:
target = TargetModel(model_type=TargetType.MBF_LARGE_V2).load('weights/backbone/mbf_large_v2.pt').freeze().eval()
shadow = ShadowModel(model_type=ShadowType.IDIAP).load("weights/shadow/mbf_large_v2+idiap.pt").set_target(target)
model = ReconstructModule(target, shadow)

Loaded the checkpoint from weights/backbone/mbf_large_v2.pt
Loaded the checkpoint from weights/shadow/mbf_large_v2+idiap.pt


In [8]:
from torchvision.datasets.folder import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms as T
from PIL import Image
dataset = ImageFolder("./face", transform=T.ToTensor())
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

In [9]:
trainer = pl.Trainer(max_epochs=1, benchmark=True)
trainer.test(model, dataloader)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  5.19it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.12402106076478958
       test_score           0.5916845798492432
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.12402106076478958, 'test_score': 0.5916845798492432}]